In [1]:
#Import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout ,Input 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [2]:
# Directory paths
age_train_dir = 'dataset/Age/train'
age_validation_dir = 'dataset/Age/test'

# Image data generators
age_train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

age_train_generator = age_train_datagen.flow_from_directory(
    age_train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

age_validation_generator = age_train_datagen.flow_from_directory(
    age_validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1697 images belonging to 51 classes.
Found 125 images belonging to 51 classes.


In [15]:
# This is to enhance the accuracy of the age model

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import RandomSearch
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras_tuner.engine.hyperparameters import HyperParameters
import os
# Verify number of classes
age_num_classes = age_train_generator.num_classes
print(f'Number of classes: {age_num_classes}')

# Pre-trained model
age_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150,150, 3))

# Fine-tune the model
age_base_model.trainable = True
fine_tune_at = 100
for layer in age_base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Function to build the model (for hyperparameter tuning)
def age_build_model(hp):
    age_model = Sequential([
        age_base_model,
        GlobalAveragePooling2D(),
        Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu'),
        Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(age_num_classes, activation='softmax')
    ])
    
    age_model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return age_model

# Hyperparameter tuning
tuner = RandomSearch(
    age_build_model,
    objective='val_accuracy',
    max_trials=12,
    executions_per_trial=2,
    directory='my_dir_task3',
    project_name='Age_model'
)

# Perform the search
tuner.search(age_train_generator, validation_data=age_validation_generator, epochs=10)

# Get the best model
best_age_model = tuner.get_best_models(num_models=1)[0]

# Further train the best model
best_age_model.fit(age_train_generator, validation_data=age_validation_generator, epochs=10)

# Save the best model
best_age_model.save('best_age_model.keras')

# Evaluate the best model
age_test_datagen = ImageDataGenerator(rescale=1./255)
age_test_generator = age_test_datagen.flow_from_directory(age_validation_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')

test_loss, test_accuracy = best_age_model.evaluate(age_test_generator)
print(f'Best Age Model Test Accuracy: {test_accuracy:.2f}')
print(f'Best Age Model Test Loss: {test_loss:.2f}')


Number of classes: 51


C:\Users\admin\AppData\Local\Temp\ipykernel_2740\2528729424.py:14: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  age_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150,150, 3))


Reloading Tuner from my_dir_task3\Age_model\tuner0.json
Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 59s 713ms/step - accuracy: 0.9919 - loss: 0.0602 - val_accuracy: 0.1680 - val_loss: 4.0469
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 675ms/step - accuracy: 0.9899 - loss: 0.0382 - val_accuracy: 0.1600 - val_loss: 4.1501
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 672ms/step - accuracy: 0.9929 - loss: 0.0311 - val_accuracy: 0.1600 - val_loss: 4.1572
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 673ms/step - accuracy: 0.9915 - loss: 0.0355 - val_accuracy: 0.1520 - val_loss: 3.9818
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 673ms/step - accuracy: 0.9906 - loss: 0.0391 - val_accuracy: 0.1600 - val_loss: 4.0574
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 675ms/step - accuracy: 0.9935 - loss: 0.0251 - val_accuracy: 0.1680 - val_loss: 4.2192
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 671ms/step - accuracy: 0.9927 - loss: 0.0217 - val_accuracy: 0.1600 - val_loss: 4.2299
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s

# emotion


In [3]:
# Directory paths
emotion_train_dir = 'dataset/Emotion/train'
emotion_validation_dir = 'dataset/Emotion/test'

# Image data generators
emotion_train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

emotion_train_generator = emotion_train_datagen.flow_from_directory(
    emotion_train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

emotion_validation_generator = emotion_train_datagen.flow_from_directory(
    emotion_validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 5068 images belonging to 6 classes.
Found 675 images belonging to 6 classes.


In [4]:
emotion_model = Sequential([
        Input(shape=(150, 150, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(6, activation='softmax') 
    ])

emotion_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])


In [5]:
history_emotion = emotion_model.fit(
    emotion_train_generator,
    epochs=10,
    validation_data=emotion_validation_generator
)


Epoch 1/10


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


159/159 ━━━━━━━━━━━━━━━━━━━━ 153s 922ms/step - accuracy: 0.1916 - loss: 1.9243 - val_accuracy: 0.1793 - val_loss: 1.7012
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 150s 942ms/step - accuracy: 0.2476 - loss: 1.7122 - val_accuracy: 0.3452 - val_loss: 1.6254
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 146s 917ms/step - accuracy: 0.3296 - loss: 1.6204 - val_accuracy: 0.3304 - val_loss: 1.6298
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 150s 946ms/step - accuracy: 0.4088 - loss: 1.4974 - val_accuracy: 0.3467 - val_loss: 1.6507
Epoch 5/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 132s 824ms/step - accuracy: 0.5124 - loss: 1.2700 - val_accuracy: 0.3526 - val_loss: 1.6505
Epoch 6/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 127s 799ms/step - accuracy: 0.6283 - loss: 1.0152 - val_accuracy: 0.3719 - val_loss: 1.7814
Epoch 7/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 124s 778ms/step - accuracy: 0.7361 - loss: 0.7312 - val_accuracy: 0.3659 - val_loss: 2.1496
Epoch 8/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 113s 711ms/step - accuracy: 0.8225 - loss: 0.51

In [31]:
# Age recognition evaluation
age_loss, age_accuracy = best_age_model.evaluate(age_validation_generator)
print(f'Age recognition accuracy: {age_accuracy * 100:.2f}%')

# Emotion detection evaluation
emotion_loss, emotion_accuracy = emotion_model.evaluate(emotion_validation_generator)
print(f'Emotion detection accuracy: {emotion_accuracy * 100:.2f}%')


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 479ms/step - accuracy: 0.1346 - loss: 4.3223
Age recognition accuracy: 13.60%
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 0.1223 - loss: 1.9505
Emotion detection accuracy: 12.69%


# Nationality


In [18]:
# Directory paths
nation_train_dir = 'dataset/Nationality/train'
nation_validation_dir = 'dataset/Nationality/test'

# Image data generators
nation_train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

nation_train_generator = nation_train_datagen.flow_from_directory(
    nation_train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

nation_validation_generator = nation_train_datagen.flow_from_directory(
    nation_validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1385 images belonging to 4 classes.
Found 49 images belonging to 4 classes.


In [19]:
nation_model = Sequential([
        Input(shape=(150, 150, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  
    ])

nation_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])


In [20]:
history_nation = nation_model.fit(
    nation_train_generator,
    epochs=10,
    validation_data=nation_validation_generator
)


Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 818ms/step - accuracy: 0.6493 - loss: 1.0402 - val_accuracy: 0.5306 - val_loss: 1.1387
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 682ms/step - accuracy: 0.7001 - loss: 0.8255 - val_accuracy: 0.5306 - val_loss: 1.1095
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 675ms/step - accuracy: 0.7136 - loss: 0.7384 - val_accuracy: 0.5102 - val_loss: 1.2349
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 670ms/step - accuracy: 0.7667 - loss: 0.6444 - val_accuracy: 0.5510 - val_loss: 1.6767
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 677ms/step - accuracy: 0.7680 - loss: 0.5676 - val_accuracy: 0.5714 - val_loss: 1.5454
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 678ms/step - accuracy: 0.8167 - loss: 0.4642 - val_accuracy: 0.5306 - val_loss: 1.8626
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 688ms/step - accuracy: 0.8510 - loss: 0.3791 - val_accuracy: 0.5102 - val_loss: 2.0892
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 685ms/step - accuracy: 0.8717 - loss: 0.3245 - val_accu

# Dress


In [21]:
# Directory paths
dress_train_dir = 'dataset/Dress/train'
dress_validation_dir = 'dataset/Dress/test'

# Image data generators
dress_train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

dress_train_generator = dress_train_datagen.flow_from_directory(
    dress_train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

dress_validation_generator = dress_train_datagen.flow_from_directory(
    dress_validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 200 images belonging to 10 classes.
Found 2 images belonging to 10 classes.


In [24]:
dress_model = Sequential([
        Input(shape=(150, 150, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')  
    ])

dress_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])


In [25]:
history_dress = dress_model.fit(
    dress_train_generator,
    epochs=10,
    validation_data=dress_validation_generator
)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 724ms/step - accuracy: 0.2111 - loss: 2.1516 - val_accuracy: 0.5000 - val_loss: 1.4266
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 638ms/step - accuracy: 0.6131 - loss: 1.0899 - val_accuracy: 0.5000 - val_loss: 1.2900
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 662ms/step - accuracy: 0.7686 - loss: 0.6669 - val_accuracy: 1.0000 - val_loss: 0.0496
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 602ms/step - accuracy: 0.8125 - loss: 0.4514 - val_accuracy: 1.0000 - val_loss: 0.0032
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 642ms/step - accuracy: 0.8811 - loss: 0.2817 - val_accuracy: 1.0000 - val_loss: 2.3539e-04
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 635ms/step - accuracy: 0.9119 - loss: 0.2776 - val_accuracy: 1.0000 - val_loss: 2.6179e-04
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 605ms/step - accuracy: 0.8711 - loss: 0.4257 - val_accuracy: 1.0000 - val_loss: 6.2808e-04
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 644ms/step - accuracy: 0.8004 - loss: 0.6478 - val_accuracy: 1.000

In [26]:
# nationality evaluation
nation_loss, nation_accuracy = nation_model.evaluate(nation_validation_generator)
print(f'Nation recognition accuracy: {nation_accuracy * 100:.2f}%')

# dress evaluation
dress_loss, dress_accuracy = dress_model.evaluate(dress_validation_generator)
print(f'Dress detection accuracy: {dress_accuracy * 100:.2f}%')


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.5412 - loss: 3.2854
Nation recognition accuracy: 53.06%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 8.7861e-04
Dress detection accuracy: 100.00%


# saving model


In [32]:
# Save the age model
best_age_model.save('age_model.keras')

# Save the emotion detection model
emotion_model.save('emotion_detection_model.keras')

# Save the nation model
nation_model.save('nation_model.keras')

# Save the dress color detection model
dress_model.save('dress_model.keras')


In [33]:
# for making classes 
import json

# Assuming train_generator is your training data generator
nationality_class_indices = nation_train_generator.class_indices
emotion_class_indices = emotion_train_generator.class_indices
dress_color_class_indices = dress_train_generator.class_indices

# Save to JSON files
with open('nationality_class_indices.json', 'w') as f:
    json.dump(nationality_class_indices, f)

with open('emotion_class_indices.json', 'w') as f:
    json.dump(emotion_class_indices, f)

with open('dress_color_class_indices.json', 'w') as f:
    json.dump(dress_color_class_indices, f)
